In [1]:
# Magic functions
%matplotlib
%load_ext autoreload
%autoreload 2

# %%javascript
# IPython.OutputArea.prototype._should_scroll = function(lines) {
#     return false;
# }

Using matplotlib backend: TkAgg


In [2]:
from calculate_zeta_t_lims import calculate_zeta_t_lims
from calculate_zeta_sp_roots import calculate_zeta_sp_roots
import importlib
import numpy as np
from plot_K_L import plot_K_L
import sys


## Constants
lambs = (0.5, 1.0)
zeta_t_abs_lim = 5
zeta_t_steps = 20 # defines scale, true number is ~4 times larger
n_pi = 4
# K_L_level = 1 # not used
n = 1000


# Calculate zeta_t border for lambda > 0
n_0 = n + n_pi
[zeta_t_internal_lims, bl_lims_exist] = calculate_zeta_t_lims(n_0, n, n_pi)

zeta_t_points_abs = []
if bl_lims_exist:
    # If limits exist, there are two disconnected regions
    # A decreasing sequence
    zeta_t_points_abs = np.linspace(zeta_t_abs_lim, zeta_t_internal_lims[1], zeta_t_steps)
    
# else:
#     # There is one connected region
#     zeta_t_points = np.linspace(-zeta_t_abs_lim, zeta_t_abs_lim, zeta_t_steps)
print("Zeta_t limits: ", zeta_t_internal_lims)



# Calculate roots
lambs_count = len(lambs)
zeta_sp_roots_size = (lambs_count, 2 * zeta_t_steps - 1)
zeta_sp_roots = np.zeros(zeta_sp_roots_size, dtype = np.float)

for lamb_ind in range(lambs_count):
    lamb = lambs[lamb_ind]

    # Not on zeta_t borders
    for zeta_ind in range(zeta_t_steps - 1):

        zeta_t = zeta_t_points_abs[zeta_ind]
#         print(zeta_t)
        cur_roots = calculate_zeta_sp_roots(zeta_t, lamb, n_0, n, n_pi)
        zeta_sp_roots[lamb_ind, zeta_ind] = cur_roots[0]
        zeta_sp_roots[lamb_ind, -1 - zeta_ind] = cur_roots[1]
        
    # Treat the middle point in a special way
    zeta_sp_roots[lamb_ind, zeta_t_steps - 1] = zeta_t_points_abs[-1] / lamb
    # zeta_sp_roots[-1, lamb_ind, :] = zeta_t_points[-1] / lamb

print(zeta_t_points_abs)
print(zeta_sp_roots[0, :])

Zeta_t limits:  [-0.96485749  0.96485749]
[5.         4.78762408 4.57524816 4.36287223 4.15049631 3.93812039
 3.72574447 3.51336855 3.30099263 3.0886167  2.87624078 2.66386486
 2.45148894 2.23911302 2.02673709 1.81436117 1.60198525 1.38960933
 1.17723341 0.96485749]
[ 9.38213288  8.98466241  8.58724349  8.18988396  7.7925934   7.3953836
  6.99826926  6.60126894  6.20440649  5.8077131   5.41123044  5.0150156
  4.61914934  4.22375029  3.82900163  3.43520558  3.04291046  2.65327129
  2.26952002  1.92971497  2.43941361  2.90516603  3.36503054  3.82223911
  4.27794675  4.73270177  5.18680641  5.64044384  6.09373269  6.54675371
  6.99956401  7.45220525  7.90470862  8.35709796  8.80939185  9.26160498
  9.71374914 10.1658339  10.61786712]


In [3]:
# Plot
plot_K_L(zeta_t_points_abs, zeta_sp_roots)

[5.         4.78762408 4.57524816 4.36287223 4.15049631 3.93812039
 3.72574447 3.51336855 3.30099263 3.0886167  2.87624078 2.66386486
 2.45148894 2.23911302 2.02673709 1.81436117 1.60198525 1.38960933
 1.17723341 0.96485749]
[1.17723341 1.38960933 1.60198525 1.81436117 2.02673709 2.23911302
 2.45148894 2.66386486 2.87624078 3.0886167  3.30099263 3.51336855
 3.72574447 3.93812039 4.15049631 4.36287223 4.57524816 4.78762408
 5.        ]


In [4]:
from find_ito_roots import find_ito_roots
find_ito_roots(n_0, n, n_pi)

[-3.67692399e-03 -1.37990164e-02 -4.41652938e-02 -9.47757562e-02
 -1.65630404e-01 -2.56729236e-01 -3.68072254e-01 -4.99659456e-01
 -6.51490845e-01 -8.23566418e-01 -1.01588618e+00 -1.22845012e+00
 -1.46125825e+00 -1.71431057e+00 -1.98760708e+00 -2.28114777e+00
 -2.59493265e+00 -2.92896171e+00 -3.28323497e+00 -3.65775241e+00
 -4.05251404e+00 -4.46751985e+00 -4.90276986e+00 -5.35826406e+00
 -5.83400245e+00 -6.32998503e+00 -6.84621179e+00 -7.38268275e+00
 -7.93939791e+00 -8.51635725e+00 -9.11356079e+00 -9.73100852e+00
 -1.03687004e+01 -1.10266366e+01 -1.17048169e+01 -1.24032414e+01
 -1.31219101e+01 -1.38608230e+01 -1.46199801e+01 -1.53993814e+01
 -1.61990269e+01 -1.70189166e+01 -1.78590505e+01 -1.87194286e+01
 -1.96000509e+01 -2.05009174e+01 -2.14220281e+01 -2.23633830e+01
 -2.33249821e+01 -2.43068255e+01 -2.53089130e+01 -2.63312447e+01
 -2.73738207e+01 -2.84366408e+01 -2.95197052e+01 -3.06230138e+01
 -3.17465666e+01 -3.28903636e+01 -3.40544048e+01 -3.52386902e+01
 -3.64432198e+01 -3.76679

In [17]:
from calculate_marginalized_integral import calculate_marginalized_integral
calculate_marginalized_integral([0.5], [1], 10, 4)

[(0.742216855240441, 8.240262419528144e-15)]